In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import ast
import json
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from functools import partial



In [4]:
# Solution is kept trivial and highly inefficient on purpose as it's provided
# purely as an example which should be straightforward to beat by anyone
def convert_dataset(dataset):
  examples = []
  for blob in dataset['txData']:
    txData = ast.literal_eval(blob)
    examples.append([
      int(txData['from'], 0) % (2 ** 30),
      (int(txData['to'], 0) if txData['to'] is not None else 0) % (2 ** 30),
      int(txData['gas'], 0),
      int(txData['gasPrice'], 0),
      (int(txData['input'][:10], 0) if txData['input'] != '0x' else 0) % (2 ** 30),
      int(txData['nonce'], 0)
    ])
  return np.array(examples)

def categorical_label(dataset):
  txTrace = dataset['txTrace']
  print(txTrace)
  err_le = LabelEncoder()
  err_label = err_le.fit_transform(txTrace['error'])
  err_ohe = OneHotEncoder()
  dataset['errorLabel'] = err_label

  err_feature_arr = err_ohe.fit_transform(dataset[['errorLabel']]).toarray()
  error = err_feature_arr.flatten()
  print((err_feature_arr))
  err_feature_label = list(err_le.classes_)
  print(err_feature_label)
  err_feature = pd.DataFrame(err_feature_arr.astype(int).astype(str))
  err_feature['error'] = err_feature.stack().groupby(level=0).apply(''.join)
  # err_feature['error'] = err_feature[err_feature_label].agg(''.join, axis=1)
  print(err_feature['error'].dtype)
  return err_feature['error']

def feature_data(dataset):
  examples = []

  
  for dat, trc in zip(dataset['txData'], dataset['txTrace']):  
    data = ast.literal_eval(dat)
    trace = ast.literal_eval(trc)
    examples.append([
      int(data['blockNumber'], 0),
      int(data['from'], 0) % (2 ** 30),
      (int(data['to'], 0) if data['to'] is not None else 0) % (2 ** 30),
      int(data['gas'], 0),
      ((int(trace['gasUsed'], 0) + 21000) *int(data['gasPrice'], 0)),
      (int(data['input'][:10], 0) if data['input'] != '0x' else 0) % (2 ** 30),
      int(data['nonce'], 0)
    ])
  return np.array(examples)


In [5]:
test =  pd.read_csv('test.csv', nrows=22)

# test = pd.concat([test.drop(['txData', 'txTrace'], axis=1), pd.json_normalize(test['txData']), pd.json_normalize(test['txTrace'])], axis=1)
# test.columns = ['txHash', 'blockNumber', 'from', 'to', 'gas', 'gasPrice', 'input',
#        'nonce', 'publicKey', 'value', 'type', 'from', 'to', 'value', 'gas1',
#        'input', 'output', 'gasUsed', 'blockNumber', 'transactionHash', 'calls',
#        'error']

# print(np.unique(test['error'].astype(str)))
# print(str(test['calls'].iloc[20] ) + str(int(test['gasUsed'].iloc[20], 0) ) + '  nonce  ' + str(int(test['nonce'].iloc[20], 0)) + '  gasPrice  ' + str(int(test['gas'].iloc[20], 0)) + ' ' + str((int(test['gasUsed'].iloc[20], 0) + 21000 )*int(test['gasPrice'].iloc[20], 0)/10**18) )
# print(test['gasUsed'].astype(str).apply(int, base=0) if test['gasUsed'].astype(str) != 'nan' else 0)
print(feature_data(test))
# err_le = LabelEncoder()
# err_ohe = OneHotEncoder()

# err_label = err_le.fit_transform(test['error'])
# test['Error_label'] = err_label

# err_feature_arr = err_ohe.fit_transform(test[['Error_label']]).toarray()
# err_feature_label = list(err_le.classes_)
# err_feature = pd.DataFrame(err_feature_arr, columns=err_feature_label)

# feature = pd.concat([test, err_feature], axis=1)
# print(feature.head())

0     {'type': 'CALL', 'from': '0xa42830ee059c77caf8...
1     {'type': 'CALL', 'from': '0x04296f2966f44fe26d...
2     {'type': 'CALL', 'from': '0x5988e6182834aa9d02...
3     {'type': 'CALL', 'from': '0x15f09a042d2e3c05c7...
4     {'type': 'CALL', 'from': '0xfde69621578dbbaafb...
5     {'type': 'CALL', 'from': '0x90db5ec42748fba62f...
6     {'type': 'CALL', 'from': '0x7879c1ee409392c050...
7     {'type': 'CALL', 'from': '0x00192fb10df37c9fb2...
8     {'type': 'CALL', 'from': '0x00192fb10df37c9fb2...
9     {'type': 'CALL', 'from': '0x0000a0756737268a63...
10    {'type': 'CALL', 'from': '0x46f43535c50edfc78c...
11    {'type': 'CALL', 'from': '0x0d0b715a705c88adf7...
12    {'type': 'CALL', 'from': '0x75e89d5979e4f6fba9...
13    {'type': 'CALL', 'from': '0xd24400ae8bfebb18ca...
14    {'type': 'CALL', 'from': '0x62af1537f9b2c9e6d9...
15    {'type': 'CALL', 'from': '0xd60dfa5ce2bdbe9ad5...
16    {'type': 'CALL', 'from': '0x494f694a03f8c7198a...
17    {'type': 'CALL', 'from': '0x945731c3791ed8

KeyError: 'error'

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
testFeatures = feature_data(test)
 
binaryModel = xgb.XGBClassifier(n_estimators=50)
binaryModel.fit(feature_data(train), train['Label0'])
print(binaryModel.feature_importances_)
binaryPredictions = binaryModel.predict_proba(testFeatures)[:, 1]
 
regressionModel = xgb.XGBRegressor(n_estimators=50)
regressionModel.fit(
  feature_data(train[train['Label0'] == True]),
  train[train['Label0'] == True]['Label1']
)
print(regressionModel.feature_importances_)
regressionPredictions = regressionModel.predict(testFeatures)
 
submission = csv.writer(open('submission.csv', 'w', encoding='UTF8'))
for x, y in zip(binaryPredictions, regressionPredictions):
  submission.writerow([x, y])

In [3]:
train = pd.read_csv('train.csv', converters={'txData': ast.literal_eval, 'txTrace': ast.literal_eval})
test = pd.read_csv('test.csv',converters={'txData': ast.literal_eval, 'txTrace': ast.literal_eval})
testFeatures = feature_data(test)
print("start")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_data(train), train['Label0'], test_size=0.2) 
binaryModel = xgb.XGBClassifier(n_estimators=50)
binaryModel.fit(X_train, y_train)
print(binaryModel.feature_importances_)
binaryPredictions = binaryModel.predict_proba(testFeatures)[:, 1]
 
regressionModel = xgb.XGBRegressor(n_estimators=50)
regressionModel.fit(
  feature_data(train[train['Label0'] == True]),
  train[train['Label0'] == True]['Label1']
)
print(regressionModel.feature_importances_)
regressionPredictions = regressionModel.predict(testFeatures)
 
predictions = binaryModel.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print('accuracy %.2f', accuracy)